In [6]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AksCompute

from azureml.core.workspace import Workspace

from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Model, Datastore, Dataset

subscription_id = '6a89c234-644a-42a1-a06b-433ce3030bb9'
resource_group = 'MSUS_SA_SupplyChain'
workspace_name = 'mlw-sa-httv3'

auth = InteractiveLoginAuthentication()


ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group,auth= auth)
# ws = Workspace.get(name=workspace_name,
#                    subscription_id=subscription_id,
#                    resource_group=resource_group,auth= auth)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

datastore = ws.get_default_datastore()


Workspace name: mlw-sa-httv3
Azure region: eastus
Subscription id: 6a89c234-644a-42a1-a06b-433ce3030bb9
Resource group: MSUS_SA_SupplyChain


In this cell, we create the environment using the dockerfile definition 

In [12]:
 #Ray Cluster deployment

from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute
from azureml.core.model import Model

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
import sklearn


environment = Environment.get(workspace=ws, name="supplychainray310", version=23)
aks_target = AksCompute(ws,"sa-aks-f8v2")
inference_config = InferenceConfig(entry_script='score_kube.py', source_directory= "../source", environment=environment)
aks_service_name="supplychainrayp104"
service = AksWebservice(ws,aks_service_name)
service.delete()
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 3, scoring_timeout_ms=300000)
service = Model.deploy(ws, aks_service_name, [], inference_config, deployment_config, aks_target,overwrite=True)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.get_logs())


/tmp/ipykernel_68179/2119009105.py:23: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, aks_service_name, [], inference_config, deployment_config, aks_target,overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-15 19:41:29+00:00 Creating Container Registry if not exists.
2023-02-15 19:41:29+00:00 Registering the environment.
2023-02-15 19:41:30+00:00 Use the existing image.
2023-02-15 19:41:31+00:00 Creating resources in AKS.
2023-02-15 19:41:32+00:00 Submitting deployment to compute.
2023-02-15 19:41:32+00:00 Checking the status of deployment supplychainrayp104..
2023-02-15 19:41:54+00:00 Checking the status of inference endpoint supplychainrayp104.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
/bin/bash: /azureml-envs/azureml_9f599a7a28bc5c7382f9f35240507922/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_9f599a7a28bc5c7382f9f35240507922/lib/libtinfo.so.6: no version information available (required by /b

In [6]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/armoshar2/code/Supply-Chain-Management/Code/Notebooks
